## Problema número 5: Summarizing with LLM ##

### Planteo del problema: ### 

Se propone la creación de un agente que reciba un archivo Bibtex como entrada, y devuelva un informe de texto con un resumen de los documentos. Además que mencione documentos, investigadores e instituciones mas relevantes relacionados con la temática de los documentos resumidos.

### Abordaje del problema: ###

Se entendió el problema como una secuencia lineal de pasos:
1. Entrada de fichero bibtex
2. Procesamiento de fichero y extraccion de titulos
3. Descarga de documentos correspondientes a cada titulo
4. Resumir cada documento
5. Obtener topico de cada documento
6. Por cada topico obtener articulos, autores e instituciones mas relevantes relacionados con el topico
7. Reportar

### Decisiones tomadas para la solución del problema: ###

1. Se trabajará con documentos pdf. Una solución más completa deberá considerar otras alternativas.

2. En el planteo del problema se habla de "temas". Estos se van a entender más bien como tópicos. Y cada documento tendrá un solo tópico. Esto facilita la formulación de la búsqueda en Google Scholar. Una solución más completa deberá considerar documentos con varios tópicos, como las compilaciones de articulos. 

3. La fuente de autoridad para recuperar autores, articulos e instituciones mas relevantes será el ranking de Google Scholar. Esto implica que los que firman los artículos más relevantes para un tópico determinado serán también los autores más relevantes, y lo mismo ocurre con sus instituciones de pertenencia. Esta es una simplificación que en dependencia de la disciplina puede ser excesiva. Una solución más completa deberá aplicar estudios de reputación, análisis de comunidades de práctica, entre otras metodologías de la cienciometría. Además en dependencia de la disciplina se deberá considerar otros tipos de instituciones, por ejemplo regulatorias, que sin dejar de ser relevantes y con impacto en un campo no producen publicaciones académicas.

4. El reporte final será texto plano, sin formato. En una solución mas completa deberá considerarse el uso de plantillas para formatear el texto y potenciar la legibilidad del reporte.

5. Se usará la librería LangChain. Esta librería se convirtió en la fuente de aprendizaje para conocer sobre agentes y chains, además de la caja de herramientas principal. Esto conllevó a la decisión más dificil y la que tomó más tiempo asumir en el ejercicio: No se va a codificar un agente, en sentido estricto y como lo entiende LangChain. Un agente según los autores de la librería tiene un flujo de trabajo no lineal, flexible, donde el agente determina el curso de acción, y elige cuáles herramienta usar entre las que tiene a su disposición. Al entenderse el problema como un proceso lineal ya no se acomoda al flujo de trabajo de un agente, a pesar de producir las salidas deseadas.

6. Una solución más completa deberá construir un agente con una serie de herramientas a su disposición que le permita planificar adecuadamente la secuencia de acciones y cambiar dinámicamente hacia flujos alternativos de trabajo. Por ejemplo, refinar iterativamente resultados de búsqueda en Google Scholar, adaptar las estrategias para hallar autores, articulos e instituciones mas relevantes a la disciplina de la que se trate, ser más flexible en el procesamiento de la información y sus variados soportes, formatear adecuadamente el reporte al usuario, entre otras funcionalidades. 

### Estructura del notebook ###

El notebook desarrolla el abordaje planteado del problema como una secuencia lineal de pasos, y  está estructurado en 6 celdas. Además de las usuales de importar las librerías necesarias y configurar variables iniciales, el resto de las celdas son:
- Funciones para trabajar con Bibtex
- Funciones para búsqueda de informacion en Google
- Uso de LangChain
- Ciclo principal de ejecución




Importa las librerias necesarias

In [ ]:
import os
import re
import json
import requests
from bibtexparser import loads
from serpapi import GoogleSearch
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.document_loaders import PyMuPDFLoader
from langchain.chains.summarize import load_summarize_chain

Configura variables de entorno y otras variables

In [ ]:
# Llaves
serpapi_key = ''
os.environ['OPENAI_API_KEY'] = ''

# Rutas
path_bibtex_file = "test-data/items2.bib"
path_download_files = 'downloaded_texts'

# Modelo a usar y parametros fundamentales. 
# Se especifica temperatura = 0 para asegurar repetibilidad de resultados
# y se mantiene modelo definido por defecto en LangChain
llm = OpenAI(temperature=0)

Funciones para trabajar con los ficheros Bibtex

In [ ]:
# Parsea fichero Bibtex y devuelve lista de titulos
def load_bibtex(file_path):
  with open(file_path) as bibtex_file:
    bib_db = loads(bibtex_file.read())
  titles = []
  for entry in bib_db.entries:    
    title = entry.get("title", "")    
    titles.append(title)
  return titles

# Devuelve url de descarga del pdf a partir de titulo de documento
def get_url_from_title(title):
    url = None
    search_params = {
        "engine": "google_scholar",
        "q": f"{title}",
        "api_key": serpapi_key,        
        "hl": "en"
    }
    search = GoogleSearch(search_params)
    results = search.get_dict()
    if 'resources' in results['organic_results'][0]:
        if 'PDF' in str(results['organic_results'][0]['resources'][0]):
            url = results['organic_results'][0]['resources'][0]['link']
    return url

# Descarga documento a partir de url y devuelve ruta de documento 
def download_document(url, folder, title):
  # Limpia y acorta string title
  title = re.sub('[^a-zA-Z]', '', title)
  title = title[:12]
  file_name = title + '.pdf'
  # Verifica existencia de carpeta antes de descarga
  if not os.path.exists(folder):
      os.makedirs(folder)
  file_path = os.path.join(folder, file_name)
  session = requests.Session()
  response = session.get(url, stream=True, headers={'User-Agent': 'Mozilla/5.0'}, allow_redirects=True)
  # Si request fue exitoso (status code 200),
  # descarga fichero
  if response.status_code == 200:
      print('Downloading...')  
      with open(file_path, 'wb') as f:
           for chunk in response.iter_content(4096):
            f.write(chunk)
      if os.path.isfile(file_path):
            print('File downloaded successfully to '+ file_path + ' !')
      else:
            print('Download failed.')
            return None
      return file_path
  else:
      print('Download failed.')
      return None

Funciones para búsqueda de informacion bibliografica usando SerpAPI

In [ ]:
# Devuelve lista de ids de autores, y cadena con titulos y autores
# a partir de topico
def get_ids_authors_articles(topic):
    res = ''
    authors_ids = []
    search_params = {
        "engine": "google_scholar",
        "q": f"{topic}",
        "api_key": serpapi_key,
        "num": 10,
        "hl": "en"
    }
    search = GoogleSearch(search_params)
    results = search.get_dict()
    results_organic = results["organic_results"]
    # Extrae la informacion de cada articulo
    for result in results_organic:
        title = result["title"]
        publication_info = result["publication_info"]
        if 'authors' in publication_info:
            authors = [author["name"] for author in publication_info["authors"]]
            first_author_id = result['publication_info']['authors'][0]['author_id']    
            res += "Title: " + title + '/n'
            res += "Authors: " + str(authors) + '/n'
            authors_ids.append(first_author_id)        
    return authors_ids, res


# Devuelve dict autor:afiliacion a partir lista de ids de autores
def get_author_affiliations(author_ids):
    affiliations = {}
    for author_id in author_ids:
        url = f"https://serpapi.com/search.json?engine=google_scholar_author&author_id={author_id}&api_key={serpapi_key}"
        response = requests.get(url)
        data = json.loads(response.text)
        if 'affiliations' in data["author"]:
            affiliations[author_id] = data["author"]["affiliations"]
        else:
            affiliations[author_id] = None
    return affiliations

Funciones para resumir documentos, extraer topicos y reportar sobre autores, documentos e instituciones principales por topico

In [ ]:
# Devuelve resumen de documento situado en document_path 
# usando LLMChain
def summarize_document(document_path):
    loader = PyMuPDFLoader(document_path)
    docs = loader.load() 
    text = '' 
    # Chequea si el pdf devuelve un string usable
    for doc in docs:
        text += doc.page_content
    pattern = r'[a-zA-Z]'
    matches = re.findall(pattern, text)
    if bool(matches):
        chain = load_summarize_chain(llm, chain_type="map_reduce")
        summary = chain.run(docs)
        return summary
    return None 

# Devuelve topico a partir de resumen usando LLMChain
def extract_topic(summary):
    prompt = PromptTemplate(
    input_variables=["text"],
    template="From the abstract in this text extract its main topic: {text}",
)
    chain = LLMChain(llm=llm, prompt=prompt)
    topic = chain.run(summary)
    return topic

# Devuelve 5 principales autores a partir de cadena con informacion
# de autores y articulos usando LLMChain, para manejar adecuadamente 
# ambiguedades, en los nombres, repeticiones, etc
def extract_main_authors(authors_and_articles):
    prompt = PromptTemplate(
    input_variables=["text"],
    template="From the following text extract the top five authors. Each author in a new line: {text}",
)
    chain = LLMChain(llm=llm, prompt=prompt)
    main_authors = chain.run(authors_and_articles)
    return main_authors

# Devuelve 5 principales articulos a partir de cadena con informacion
# de autores y articulos usando LLMChain, para manejar adecuadamente
# titulos repetidos, con pequeñas variaciones, etc
def extract_main_articles(authors_and_articles):
    prompt = PromptTemplate(
    input_variables=["text"],
    template="From the following text extract the top five titles. Mention its authors: {text}",
)
    chain = LLMChain(llm=llm, prompt=prompt)
    main_articles = chain.run(authors_and_articles)
    return main_articles

# Devuelve principales centros e instituciones para determinado 
# topico a partir de las afiliaciones de autores
def report_main_centers_and_institutions(affiliations):
    prompt = PromptTemplate(
    input_variables=["text"],
    template="Report only institution names from the following text. Ignore personal names, titles and roles. Also ignore incomplete information. Each institution in a new line: {text}",
)
    chain = LLMChain(llm=llm, prompt=prompt)
    top_centers = chain.run(str(affiliations))
    return top_centers

Ciclo principal

In [ ]:
# Devuelve lista de titulos a partir de fichero Bibtex en path_bibtex_file
titles = load_bibtex(path_bibtex_file)

for title in titles:
    # Obten url de descarga de version pdf del titulo
    url = get_url_from_title(title)

    # Si no se pudo obtener version pdf del documento pasa al siguiente titulo
    if url == None:
        print('Impossible to find pdf of: ' + title)
        continue
    
    # Descarga el documento
    document_path = download_document(url, path_download_files, title)

    # Si no se pudo descargar el documento pasa al siguiente titulo
    if document_path == None:
        print('Impossible to download pdf of: ' + title)
        continue
    
    # Resume el documento
    summary = summarize_document(document_path)

    # Si no se pudo extraer texto del documento pasa al siguiente titulo
    if summary == None:
        print('Impossible to extract text from pdf: ' + title)
        continue
    
    # Extrae el topico del documento
    topic = extract_topic(summary)
    
    # Obten la informacion sobre principales autores y articulos en
    # el topico
    author_ids, authors_and_articles = get_ids_authors_articles(topic)

    # Obten principales autores
    main_authors = extract_main_authors(authors_and_articles) 

    # Obten principales articulos
    main_articles = extract_main_articles(authors_and_articles) 
    
    # Obten afiliaciones de cada autor
    affiliations = get_author_affiliations(author_ids)
    
    # Extrae principales centros e instituciones de las afiliaciones
    top_centers = report_main_centers_and_institutions(str(affiliations))
    
    # Imprime la informacion obtenida para este titulo
    print('')
    print(f'Title: {title}')
    print('Abstract:')
    print(summary)
    print(topic)
    print('Main_authors in the topic: ')
    print(main_authors)
    print('')
    print('Main documents in the topic: ')
    print(main_articles)
    print('')
    print('Main centers and institutions: ')
    print(top_centers)
    print('')
    print('--------------------------------------')